# Convolutional Attention Block

Propuesta alternativa al CoordinateAttention.

Se utiliza kernels de distintos tamaños para obtener descriptores horizontales y verticales, el propósito es obtener descriptores que definan la información espacial de forma complementaria para luego unificar dicha información mediante la adición (o concatenación) de ambos descriptores.

De todas formas, no se puede ignorar la información de los canales de cada input. Estos podrían afectar drásticamente a la los descriptores espaciales. Por este motivo, se han aplicado DepthWise Separable convolutions para que cada descriptor no se base toda su información teniendo en cuenta todos los canales, sino un subconjunto de ellos. (Es necesario desarrollar esto).

In [ ]:
import torch
from torch import nn

### Attention Block V1

In [ ]:
class ConvolutionalAttentionBlock(nn.Module):
    def __init__(self, img_size: tuple, in_channels: int, reduction_rate: int, groups=False, bias=True) -> None:
        super(ConvolutionalAttentionBlock, self).__init__()
        mip = max(8, in_channels // reduction_rate)
        
        self.squeeze_h = nn.Sequential(
            nn.AdaptiveAvgPool2d((None, 1)),
            nn.Conv2d(in_channels, mip, 1, bias=False),
            nn.BatchNorm2d(mip),
            nn.SiLU()
        )

        self.squeeze_w = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, None)),
            nn.Conv2d(in_channels, mip, 1, bias=False),
            nn.BatchNorm2d(mip),
            nn.SiLU()
        )

        self.excitation = nn.Sequential(
            nn.Conv2d(mip, in_channels, 1, bias=False),
            nn.BatchNorm2d(in_channels),
            nn.Sigmoid()
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x_h = self.squeeze_h(x) # Height descriptor shape: (C x W x 1)
        x_w = self.squeeze_w(x) # Width descriptor shape: (C x 1 x H)

        # Coordinate attention
        coordAtt = self.excitation(x_h+x_w)
        # TODO: Concatenate x_h and x_w
        
        return coordAtt  

### Attention Block V2

In [ ]:
class ConvolutionalAttentionBlock(nn.Module):
    def __init__(self, img_size: tuple, in_channels: int, reduction_rate: int, groups=True, bias=False) -> None:
        super(ConvolutionalAttentionBlock, self).__init__()
        mip = max(8, in_channels // reduction_rate)
        H, W = img_size

        self.conv_h = nn.Sequential(
            nn.Conv2d(in_channels, mip, (1, W), bias=bias, groups=mip if groups else 1),
            nn.BatchNorm2d(mip),
            nn.SiLU()
        )

        self.conv_w = nn.Sequential(
            nn.Conv2d(in_channels, mip, (H, 1), bias=bias, groups=mip if groups else 1),
            nn.BatchNorm2d(mip),
            nn.SiLU()
        )
        
        self.att = nn.Sequential(
            nn.Conv2d(mip, in_channels, 1, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x_h = self.conv_h(x) # Height descriptor
        x_w = self.conv_w(x) # Width descriptor

        # Coordinate attention
        coordAtt = self.att(x_h+x_w)
        # TODO: Concatenate x_h and x_w
        
        return coordAtt  

# RESNet-18

## Residual Block with Attention

In [ ]:
from torchvision.models.resnet import BasicBlock

class ResAttentionBlock(BasicBlock):
    def __init__(self, img_size: tuple, inplanes:int, planes:int, stride=1, downsample=None, 
                att_reduction=8, att_groups=True, att_bias=True, **kargs):
                 
        super(ResAttentionBlock, self).__init__(inplanes, planes, stride, downsample)

        self.attention = ConvolutionalAttentionBlock(img_size, planes, att_reduction, 
                    groups=att_groups, bias=att_bias)

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        att = self.attention(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        res = self.relu((att*out) + identity)
        return res

## ResNet with Attention

In [ ]:
from torchvision.models.resnet import conv1x1
from torchvision.models import ResNet
import numpy as np

class ResNet_Attention(ResNet):
    def __init__(self, img_size:tuple, block:nn.Module, layers:list, num_classes=1000, **kargs):
        super(ResNet_Attention, self).__init__(BasicBlock, layers, num_classes, kargs)

        if not isinstance(img_size, np.ndarray):
            img_size = np.array(img_size)

        self.inplanes = 64 # Because in super init it has been set to 512
        self.layer1 = self._make_attention_layer(tuple(img_size // (2**2)), block, 64, layers[0], **kargs)
        self.layer2 = self._make_attention_layer(tuple(img_size // (2**3)), block, 128, layers[1], stride=2,
                                       dilate=False, **kargs)
        self.layer3 = self._make_attention_layer(tuple(img_size // (2**4)), block, 256, layers[2], stride=2,
                                       dilate=False, **kargs)
        self.layer4 = self._make_attention_layer(tuple(img_size // (2**5)), block, 512, layers[3], stride=2,
                                       dilate=False, **kargs)
        
    def _make_attention_layer(self, input_size, block, planes, blocks, stride=1, dilate=False, **kargs):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(input_size, self.inplanes, planes, stride, downsample, 
                    groups=self.groups, base_width=self.base_width, dilation=previous_dilation,
                    norm_layer=norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(input_size, self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)



In [ ]:
from torchvision.models.resnet import BasicBlock

# ResNet-18 config
# https://github.com/pytorch/vision/blob/28557e0cfe9113a5285330542264f03e4ba74535/torchvision/models/resnet.py#L649-L670
resnet_18 = ResNet_Attention((128,128), ResAttentionBlock, [2,2,2,2], num_classes=1000)
result = resnet_18(torch.rand(1,3,128,128))

# Training

In [ ]:
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, ToTensor, Resize

transform = Compose([Resize(128), ToTensor()])
train_dataset = CIFAR10('dataset/', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)

test_dataset = CIFAR10('dataset/', train=False, transform=transform, download=False)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

img_size = (128,128)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

def train(model, n_epoch, train_loader, test_loader, exp_name):
    # optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=.9)
    criterion = nn.CrossEntropyLoss()

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)

    tb_writer = SummaryWriter('log/{}'.format(exp_name))
    running_avg_accuracy = 0
    step = 0

    epoch_iterator = tqdm(
            range(n_epoch),
            leave=True,
            unit="epoch",
            postfix={"tls": "%.4f" % 1},
        )

    for epoch in epoch_iterator:
        model.eval()
        total = 0
        correct = 0

        model.train()
        for idx, (inputs, targets) in enumerate(train_loader):
            optimizer.zero_grad()

            inputs = inputs.to(device)
            targets = targets.to(device)
            pred = model(inputs)

            loss = criterion(pred, targets)
            loss.backward()
            optimizer.step()

            if idx % 250 == 0:
                model.eval()
                pred = model(inputs)
                predict = torch.argmax(pred, 1)
                total = targets.size(0)
                correct = torch.eq(predict, targets).sum().double().item()
                accuracy = correct / total
                running_avg_accuracy = 0.6*running_avg_accuracy + 0.4*accuracy
                tb_writer.add_scalar('train/loss', loss.item(), step)
                tb_writer.add_scalar('train/accuracy', accuracy, step)
                tb_writer.add_scalar('train/running_avg_accuracy', running_avg_accuracy, step)
                step += 1

                epoch_iterator.set_postfix(tls="%.4f" % loss.item())      

        with torch.no_grad():
            for i, data in enumerate(test_loader, 0):
                    images_test, labels_test = data
                    images_test, labels_test = images_test.to(device), labels_test.to(device)
                    pred_test = model(images_test)
                    predict = torch.argmax(pred_test, 1)
                    total += labels_test.size(0)
                    correct += torch.eq(predict, labels_test).sum().double().item()
                    
            tb_writer.add_scalar('test/accuracy', correct/total, epoch)

    return model

In [ ]:
from torchvision.models import resnet18
resnet_18 = resnet18(num_classes=10)
train(resnet_18, 50, train_loader, test_loader, 'ConvAttention/CIFAR/original_resnet18')

In [ ]:
resnet_18 = ResNet_Attention((128,128), ResAttentionBlock, [2, 2, 2, 2], num_classes=10)
train(resnet_18, 50, train_loader, test_loader, 'ConvAttention/CIFAR/attention_v2_resnet18')